<a href="https://colab.research.google.com/github/GiammarcoBozzelli/DSML/blob/wip/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset
url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/training_data.csv"
# augmented_df = pd.read_csv(url)
data = pd.read_csv(url)

In [ ]:
# augmented_df = pd.read_csv('https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/augmented_dataset.csv')

In [2]:
 %%capture
 !pip install transformers[torch] accelerate -U

## Distilbert

In [27]:
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler
import numpy as np
import nltk
from nltk.corpus import wordnet
from random import randint
import random
import string

# Download necessary NLTK data
nltk.download('punkt')

# Load the dataset
url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/training_data.csv"
data = pd.read_csv(url)

# Map labels to integers
label_mapping = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}
data['difficulty'] = data['difficulty'].map(label_mapping)

# Feature Engineering Functions
def get_sentence_length(sentence):
    return len(sentence)

def get_word_count(sentence):
    words = nltk.word_tokenize(sentence)
    return len(words)

def get_avg_word_length(sentence):
    words = nltk.word_tokenize(sentence)
    return np.mean([len(word) for word in words])

def count_punctuation(sentence):
    return sum([1 for char in sentence if char in string.punctuation])

# Extract Features
data['sentence_length'] = data['sentence'].apply(get_sentence_length)
data['word_count'] = data['sentence'].apply(get_word_count)
data['avg_word_length'] = data['sentence'].apply(get_avg_word_length)
data['punctuation_count'] = data['sentence'].apply(count_punctuation)

# Standardize the features
scaler = StandardScaler()
feature_cols = ['sentence_length', 'word_count', 'avg_word_length', 'punctuation_count']
data[feature_cols] = scaler.fit_transform(data[feature_cols])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data, data['difficulty'], test_size=0.2, random_state=42)

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

train_encodings = tokenizer(X_train['sentence'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(X_test['sentence'].tolist(), truncation=True, padding=True)

class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None, features=None):
        self.encodings = encodings
        self.labels = labels
        self.features = features

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        if self.features is not None:
            item['features'] = torch.tensor(self.features[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Set the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')



# Cross-validation with KFold
kf = KFold(n_splits=3, shuffle=True, random_state=42)
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"Training fold {fold + 1}")

    # Prepare training and validation datasets
    train_data = data.iloc[train_idx]
    val_data = data.iloc[val_idx]
    # Load pre-trained model
    model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=6).to(device)

    train_encodings_fold = tokenizer(train_data['sentence'].tolist(), truncation=True, padding=True)
    val_encodings_fold = tokenizer(val_data['sentence'].tolist(), truncation=True, padding=True)

    train_dataset = ClassificationDataset(train_encodings_fold, train_data['difficulty'].tolist(), train_data[feature_cols].values)
    val_dataset = ClassificationDataset(val_encodings_fold, val_data['difficulty'].tolist(), val_data[feature_cols].values)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f'./results_fold_{fold + 1}',
        num_train_epochs=8,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=1000,
        weight_decay=0.0015,
        logging_dir=f'./logs_fold_{fold + 1}',
        logging_steps=20,
        evaluation_strategy="epoch",
        learning_rate=0.00005,
        fp16=True
    )

    # Define trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    predictions, label_ids, metrics = trainer.predict(val_dataset)
    preds = predictions.argmax(-1)

    # Calculate metrics
    accuracy = accuracy_score(val_data['difficulty'], preds)
    precision = precision_score(val_data['difficulty'], preds, average='weighted')
    recall = recall_score(val_data['difficulty'], preds, average='weighted')
    f1 = f1_score(val_data['difficulty'], preds, average='weighted')

    # Store metrics
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)

    # Print the results
    print(f'Fold {fold + 1} - Accuracy: {accuracy:.2f}')
    print(f'Fold {fold + 1} - Precision: {precision:.2f}')
    print(f'Fold {fold + 1} - Recall: {recall:.2f}')
    print(f'Fold {fold + 1} - F1 Score: {f1:.2f}')

# Print the average results across folds
print(f'Average Accuracy: {np.mean(accuracy_list):.2f}')
print(f'Average Precision: {np.mean(precision_list):.2f}')
print(f'Average Recall: {np.mean(recall_list):.2f}')
print(f'Average F1 Score: {np.mean(f1_list):.2f}')

# Alternatively, print a detailed classification report for the final fold
print(classification_report(val_data['difficulty'], preds, target_names=['A1', 'A2', 'B1', 'B2', 'C1', 'C2']))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Training fold 1


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.299600,1.238442
2,1.189300,1.190658
3,1.066300,1.236999
4,0.802100,1.295088
5,0.379000,1.553977
6,0.089100,2.290314
7,0.176600,2.572154
8,0.008500,2.728797


Fold 1 - Accuracy: 0.55
Fold 1 - Precision: 0.58
Fold 1 - Recall: 0.55
Fold 1 - F1 Score: 0.55
Training fold 2


...

Epoch,Training Loss,Validation Loss
1,1.243300,1.292821
2,1.125900,1.190287
3,0.977500,1.269662
4,0.595600,1.421422
5,0.408800,1.986328
6,0.160800,2.612634
7,0.051900,2.999761
8,0.003000,3.057564


Fold 2 - Accuracy: 0.50
Fold 2 - Precision: 0.52
Fold 2 - Recall: 0.50
Fold 2 - F1 Score: 0.51
Training fold 3


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.311700,1.314149
2,1.219400,1.216400
3,1.065800,1.159325
4,0.857300,1.320384
5,0.423800,1.665996
6,0.258400,2.245732
7,0.091200,2.606750
8,0.052300,2.633737


Fold 3 - Accuracy: 0.54
Fold 3 - Precision: 0.55
Fold 3 - Recall: 0.54
Fold 3 - F1 Score: 0.54
Average Accuracy: 0.53
Average Precision: 0.55
Average Recall: 0.53
Average F1 Score: 0.53
              precision    recall  f1-score   support

          A1       0.66      0.66      0.66       205
          A2       0.50      0.49      0.50       214
          B1       0.45      0.49      0.47       216
          B2       0.53      0.51      0.52       221
          C1       0.49      0.53      0.51       208
          C2       0.66      0.58      0.62       216

    accuracy                           0.54      1280
   macro avg       0.55      0.54      0.55      1280
weighted avg       0.55      0.54      0.54      1280



In [12]:
model.save_pretrained('./distilbert')
tokenizer.save_pretrained('./distilbert')

('./distilbert/tokenizer_config.json',
 './distilbert/special_tokens_map.json',
 './distilbert/vocab.txt',
 './distilbert/added_tokens.json')

In [13]:
!zip -r ./distilbert.zip ./distilbert/

  adding: distilbert/ (stored 0%)
  adding: distilbert/config.json (deflated 52%)
  adding: distilbert/vocab.txt (deflated 45%)
  adding: distilbert/special_tokens_map.json (deflated 42%)
  adding: distilbert/model.safetensors (deflated 7%)
  adding: distilbert/tokenizer_config.json (deflated 75%)


## Camembert

In [14]:
import pandas as pd
import torch
from transformers import CamembertTokenizer,CamembertForSequenceClassification, CamembertModel, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler
import numpy as np
import nltk
from nltk.corpus import wordnet
from random import randint
import random
import string

# Download necessary NLTK data
nltk.download('punkt')

# Load the dataset
url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/training_data.csv"
data = pd.read_csv(url)

# Map labels to integers
label_mapping = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}
data['difficulty'] = data['difficulty'].map(label_mapping)

# Feature Engineering Functions
def get_sentence_length(sentence):
    return len(sentence)

def get_word_count(sentence):
    words = nltk.word_tokenize(sentence)
    return len(words)

def get_avg_word_length(sentence):
    words = nltk.word_tokenize(sentence)
    return np.mean([len(word) for word in words])

def count_punctuation(sentence):
    return sum([1 for char in sentence if char in string.punctuation])

# Extract Features
data['sentence_length'] = data['sentence'].apply(get_sentence_length)
data['word_count'] = data['sentence'].apply(get_word_count)
data['avg_word_length'] = data['sentence'].apply(get_avg_word_length)
data['punctuation_count'] = data['sentence'].apply(count_punctuation)

# Standardize the features
scaler = StandardScaler()
feature_cols = ['sentence_length', 'word_count', 'avg_word_length', 'punctuation_count']
data[feature_cols] = scaler.fit_transform(data[feature_cols])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data, data['difficulty'], test_size=0.2, random_state=42)

# Tokenization
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

train_encodings = tokenizer(X_train['sentence'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(X_test['sentence'].tolist(), truncation=True, padding=True)

class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None, features=None):
        self.encodings = encodings
        self.labels = labels
        self.features = features

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        if self.features is not None:
            item['features'] = torch.tensor(self.features[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Set the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Cross-validation with KFold
kf = KFold(n_splits=3, shuffle=True, random_state=42)
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"Training fold {fold + 1}")

    model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6).to(device)

    # Prepare training and validation datasets
    train_data = data.iloc[train_idx]
    val_data = data.iloc[val_idx]

    train_encodings_fold = tokenizer(train_data['sentence'].tolist(), truncation=True, padding=True)
    val_encodings_fold = tokenizer(val_data['sentence'].tolist(), truncation=True, padding=True)

    train_dataset = ClassificationDataset(train_encodings_fold, train_data['difficulty'].tolist(), train_data[feature_cols].values)
    val_dataset = ClassificationDataset(val_encodings_fold, val_data['difficulty'].tolist(), val_data[feature_cols].values)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f'./results_fold_{fold + 1}',
        num_train_epochs=7,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=1000,
        weight_decay=0.0015,
        logging_dir=f'./logs_fold_{fold + 1}',
        logging_steps=20,
        evaluation_strategy="epoch",
        learning_rate=0.00001,
        fp16=True
    )

    # Define trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    predictions, label_ids, metrics = trainer.predict(val_dataset)
    preds = predictions.argmax(-1)

    # Calculate metrics
    accuracy = accuracy_score(val_data['difficulty'], preds)
    precision = precision_score(val_data['difficulty'], preds, average='weighted')
    recall = recall_score(val_data['difficulty'], preds, average='weighted')
    f1 = f1_score(val_data['difficulty'], preds, average='weighted')

    # Store metrics
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)

    # Print the results
    print(f'Fold {fold + 1} - Accuracy: {accuracy:.2f}')
    print(f'Fold {fold + 1} - Precision: {precision:.2f}')
    print(f'Fold {fold + 1} - Recall: {recall:.2f}')
    print(f'Fold {fold + 1} - F1 Score: {f1:.2f}')

# Print the average results across folds
print(f'Average Accuracy: {np.mean(accuracy_list):.2f}')
print(f'Average Precision: {np.mean(precision_list):.2f}')
print(f'Average Recall: {np.mean(recall_list):.2f}')
print(f'Average F1 Score: {np.mean(f1_list):.2f}')

# Alternatively, print a detailed classification report for the final fold
print(classification_report(val_data['difficulty'], preds, target_names=['A1', 'A2', 'B1', 'B2', 'C1', 'C2']))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Training fold 1


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.766800,1.755382
2,1.553200,1.485113
3,1.348500,1.272319
4,1.183100,1.187115
5,1.001700,1.096363
6,0.962900,1.053249
7,0.916800,1.051031


Fold 1 - Accuracy: 0.57
Fold 1 - Precision: 0.57
Fold 1 - Recall: 0.57
Fold 1 - F1 Score: 0.56
Training fold 2


...

Epoch,Training Loss,Validation Loss
1,1.767000,1.769972
2,1.537200,1.496732
3,1.293400,1.270783
4,1.125000,1.209515
5,0.922000,1.176997
6,0.872600,1.182077
7,0.887000,1.186372


Fold 2 - Accuracy: 0.49
Fold 2 - Precision: 0.50
Fold 2 - Recall: 0.49
Fold 2 - F1 Score: 0.48
Training fold 3


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.769100,1.768578
2,1.564000,1.522200
3,1.321700,1.313596
4,1.199200,1.166784
5,1.055300,1.098279
6,0.939400,1.097569
7,0.825200,1.109191


Fold 3 - Accuracy: 0.52
Fold 3 - Precision: 0.52
Fold 3 - Recall: 0.52
Fold 3 - F1 Score: 0.51
Average Accuracy: 0.53
Average Precision: 0.53
Average Recall: 0.53
Average F1 Score: 0.52
              precision    recall  f1-score   support

          A1       0.61      0.81      0.70       205
          A2       0.48      0.43      0.45       214
          B1       0.46      0.48      0.47       216
          B2       0.44      0.53      0.48       221
          C1       0.42      0.29      0.35       208
          C2       0.70      0.59      0.64       216

    accuracy                           0.52      1280
   macro avg       0.52      0.52      0.51      1280
weighted avg       0.52      0.52      0.51      1280



In [16]:
model.save_pretrained('./camembert')
tokenizer.save_pretrained('./camembert')

('./camembert/tokenizer_config.json',
 './camembert/special_tokens_map.json',
 './camembert/sentencepiece.bpe.model',
 './camembert/added_tokens.json')

In [17]:
!zip -r ./camembert.zip ./camembert/

  adding: camembert/ (stored 0%)
  adding: camembert/config.json (deflated 55%)
  adding: camembert/sentencepiece.bpe.model (deflated 49%)
  adding: camembert/special_tokens_map.json (deflated 52%)
  adding: camembert/model.safetensors (deflated 12%)
  adding: camembert/tokenizer_config.json (deflated 81%)
  adding: camembert/added_tokens.json (stored 0%)


In [4]:
# from transformers import CamembertTokenizer,CamembertForSequenceClassification, CamembertModel, pipeline
# X_train, X_test, y_train, y_test = train_test_split(data, data['difficulty'], test_size=0.2, random_state=42)

# # Tokenization
# tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# train_encodings = tokenizer(X_train['sentence'].tolist(), truncation=True, padding=True)
# test_encodings = tokenizer(X_test['sentence'].tolist(), truncation=True, padding=True)

# class ClassificationDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels=None, features=None):
#         self.encodings = encodings
#         self.labels = labels
#         self.features = features

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         if self.labels is not None:
#             item['labels'] = torch.tensor(self.labels[idx])
#         if self.features is not None:
#             item['features'] = torch.tensor(self.features[idx], dtype=torch.float32)
#         return item

#     def __len__(self):
#         return len(self.encodings['input_ids'])

# # Prepare datasets
# train_dataset = ClassificationDataset(train_encodings, y_train.tolist(), X_train[feature_cols].values)
# test_dataset = ClassificationDataset(test_encodings, y_test.tolist(), X_test[feature_cols].values)

# # Set the device
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# # Load pre-trained model
# model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6).to(device)

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     warmup_steps=1000,
#     weight_decay=0.05,
#     logging_dir='./logs',
#     logging_steps=10,
#     evaluation_strategy="epoch",
#     learning_rate=0.0001,
#     fp16=True
# )

# # Define trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset
# )

# # Train the model
# trainer.train()

# # Evaluate the model
# predictions, label_ids, metrics = trainer.predict(test_dataset)
# preds = predictions.argmax(-1)

# # Calculate metrics
# accuracy = accuracy_score(y_test, preds)
# precision = precision_score(y_test, preds, average='weighted')
# recall = recall_score(y_test, preds, average='weighted')
# f1 = f1_score(y_test, preds, average='weighted')

# # Print the results
# print(f'Accuracy: {accuracy:.2f}')
# print(f'Precision: {precision:.2f}')
# print(f'Recall: {recall:.2f}')
# print(f'F1 Score: {f1:.2f}')

# # Alternatively, print a detailed classification report
# print(classification_report(y_test, preds, target_names=['A1', 'A2', 'B1', 'B2', 'C1', 'C2']))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.395600,1.331837
2,1.093200,1.205581
3,0.979200,1.151593
4,0.679100,1.085668
5,0.509100,1.268689
6,0.297300,1.464791
7,0.130900,1.695207


Accuracy: 0.56
Precision: 0.58
Recall: 0.56
F1 Score: 0.56
              precision    recall  f1-score   support

          A1       0.74      0.73      0.74       166
          A2       0.47      0.56      0.51       158
          B1       0.50      0.52      0.51       166
          B2       0.47      0.59      0.52       153
          C1       0.50      0.52      0.51       152
          C2       0.80      0.42      0.55       165

    accuracy                           0.56       960
   macro avg       0.58      0.56      0.56       960
weighted avg       0.58      0.56      0.56       960



In [18]:

!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 8.1 MB/s eta 0:00:00


## Flaubert

In [23]:
import pandas as pd
import torch
from transformers import FlaubertModel, FlaubertTokenizer,FlaubertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler
import numpy as np
import nltk
from nltk.corpus import wordnet
from random import randint
import random
import string

# Download necessary NLTK data
nltk.download('punkt')

# Load the dataset
url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/training_data.csv"
data = pd.read_csv(url)

# Map labels to integers
label_mapping = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}
data['difficulty'] = data['difficulty'].map(label_mapping)

# Feature Engineering Functions
def get_sentence_length(sentence):
    return len(sentence)

def get_word_count(sentence):
    words = nltk.word_tokenize(sentence)
    return len(words)

def get_avg_word_length(sentence):
    words = nltk.word_tokenize(sentence)
    return np.mean([len(word) for word in words])

def count_punctuation(sentence):
    return sum([1 for char in sentence if char in string.punctuation])

# Extract Features
data['sentence_length'] = data['sentence'].apply(get_sentence_length)
data['word_count'] = data['sentence'].apply(get_word_count)
data['avg_word_length'] = data['sentence'].apply(get_avg_word_length)
data['punctuation_count'] = data['sentence'].apply(count_punctuation)

# Standardize the features
scaler = StandardScaler()
feature_cols = ['sentence_length', 'word_count', 'avg_word_length', 'punctuation_count']
data[feature_cols] = scaler.fit_transform(data[feature_cols])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data, data['difficulty'], test_size=0.2, random_state=42)

# Tokenization
tokenizer = FlaubertTokenizer.from_pretrained('flaubert/flaubert_base_cased')

train_encodings = tokenizer(X_train['sentence'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(X_test['sentence'].tolist(), truncation=True, padding=True)

class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None, features=None):
        self.encodings = encodings
        self.labels = labels
        self.features = features

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        if self.features is not None:
            item['features'] = torch.tensor(self.features[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Set the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Load pre-trained model


# Cross-validation with KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"Training fold {fold + 1}")

    # Prepare training and validation datasets
    train_data = data.iloc[train_idx]
    val_data = data.iloc[val_idx]

    model = FlaubertForSequenceClassification.from_pretrained('flaubert/flaubert_base_cased', num_labels=6).to(device)

    train_encodings_fold = tokenizer(train_data['sentence'].tolist(), truncation=True, padding=True)
    val_encodings_fold = tokenizer(val_data['sentence'].tolist(), truncation=True, padding=True)

    train_dataset = ClassificationDataset(train_encodings_fold, train_data['difficulty'].tolist(), train_data[feature_cols].values)
    val_dataset = ClassificationDataset(val_encodings_fold, val_data['difficulty'].tolist(), val_data[feature_cols].values)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f'./results_fold_{fold + 1}',
        num_train_epochs=10,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=1000,
        weight_decay=0.0015,
        logging_dir=f'./logs_fold_{fold + 1}',
        logging_steps=20,
        evaluation_strategy="epoch",
        learning_rate=0.00001,
        fp16=True
    )

    # Define trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    predictions, label_ids, metrics = trainer.predict(val_dataset)
    preds = predictions.argmax(-1)

    # Calculate metrics
    accuracy = accuracy_score(val_data['difficulty'], preds)
    precision = precision_score(val_data['difficulty'], preds, average='weighted')
    recall = recall_score(val_data['difficulty'], preds, average='weighted')
    f1 = f1_score(val_data['difficulty'], preds, average='weighted')

    # Store metrics
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)

    # Print the results
    print(f'Fold {fold + 1} - Accuracy: {accuracy:.2f}')
    print(f'Fold {fold + 1} - Precision: {precision:.2f}')
    print(f'Fold {fold + 1} - Recall: {recall:.2f}')
    print(f'Fold {fold + 1} - F1 Score: {f1:.2f}')

# Print the average results across folds
print(f'Average Accuracy: {np.mean(accuracy_list):.2f}')
print(f'Average Precision: {np.mean(precision_list):.2f}')
print(f'Average Recall: {np.mean(recall_list):.2f}')
print(f'Average F1 Score: {np.mean(f1_list):.2f}')

# Alternatively, print a detailed classification report for the final fold
print(classification_report(val_data['difficulty'], preds, target_names=['A1', 'A2', 'B1', 'B2', 'C1', 'C2']))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Training fold 1


Some weights of FlaubertForSequenceClassification were not initialized from the model checkpoint at flaubert/flaubert_base_cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.772200,1.733193
2,1.622100,1.487104
3,1.146700,1.209727
4,1.239000,1.133424
5,1.045000,1.099472
6,1.027000,1.054684
7,0.939800,1.057798
8,0.883500,1.077912
9,0.803600,1.094515
10,0.651900,1.101794


Fold 1 - Accuracy: 0.59
Fold 1 - Precision: 0.59
Fold 1 - Recall: 0.59
Fold 1 - F1 Score: 0.59
Training fold 2


...

Epoch,Training Loss,Validation Loss
1,1.775700,1.671537
2,1.553800,1.471448
3,1.243400,1.244058
4,1.048100,1.152903
5,1.084600,1.112401
6,0.902800,1.119035
7,0.740300,1.150162
8,0.737000,1.175533
9,0.734400,1.180974
10,0.794200,1.206400


Fold 4 - Accuracy: 0.54
Fold 4 - Precision: 0.55
Fold 4 - Recall: 0.54
Fold 4 - F1 Score: 0.54
Training fold 5


Some weights of FlaubertForSequenceClassification were not initialized from the model checkpoint at flaubert/flaubert_base_cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.757800,1.695904
2,1.493100,1.445942
3,1.325500,1.207282
4,1.251800,1.158775
5,1.103300,1.062548
6,0.927600,1.106098
7,0.982600,1.096968
8,0.896500,1.128776
9,0.724600,1.151782
10,0.686000,1.103826


Fold 5 - Accuracy: 0.56
Fold 5 - Precision: 0.56
Fold 5 - Recall: 0.56
Fold 5 - F1 Score: 0.56
Average Accuracy: 0.56
Average Precision: 0.56
Average Recall: 0.56
Average F1 Score: 0.56
              precision    recall  f1-score   support

          A1       0.72      0.69      0.71       122
          A2       0.54      0.60      0.57       131
          B1       0.48      0.58      0.53       118
          B2       0.57      0.53      0.55       145
          C1       0.43      0.36      0.39       121
          C2       0.62      0.62      0.62       131

    accuracy                           0.56       768
   macro avg       0.56      0.56      0.56       768
weighted avg       0.56      0.56      0.56       768



In [24]:
model.save_pretrained('./flaubert')
tokenizer.save_pretrained('./flaubert')

('./flaubert/tokenizer_config.json',
 './flaubert/special_tokens_map.json',
 './flaubert/vocab.json',
 './flaubert/merges.txt',
 './flaubert/added_tokens.json')

In [25]:
!zip -r ./flaubert.zip ./flaubert/

  adding: flaubert/ (stored 0%)
  adding: flaubert/merges.txt (deflated 60%)
  adding: flaubert/vocab.json (deflated 71%)
  adding: flaubert/config.json (deflated 61%)
  adding: flaubert/special_tokens_map.json (deflated 64%)
  adding: flaubert/model.safetensors (deflated 7%)
  adding: flaubert/tokenizer_config.json (deflated 87%)


## Pipeline

In [26]:
from transformers import pipeline
unlabelled_url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/unlabelled_test_data.csv"
unlabelled_data = pd.read_csv(unlabelled_url)

# Load the saved CamemBERT model and tokenizer

distilbert_path = './distilbert'
distilbert_model = DistilBertForSequenceClassification.from_pretrained(distilbert_path)
distilbert_tokenizer = DistilBertTokenizer.from_pretrained(distilbert_path)


# Load the saved CamemBERT model and tokenizer
camembert_path = './camembert'
camembert_model = CamembertForSequenceClassification.from_pretrained(camembert_path)
camembert_tokenizer = CamembertTokenizer.from_pretrained(camembert_path)

# Load the saved Flaubert model and tokenizer
flaubert_path = './flaubert'
flaubert_model = FlaubertForSequenceClassification.from_pretrained(flaubert_path)
flaubert_tokenizer = FlaubertTokenizer.from_pretrained(flaubert_path)

# Create prediction pipelines to get probabilities
device = 0 if torch.cuda.is_available() else -1
distilbert_classifier = pipeline('text-classification', model=distilbert_model, tokenizer=distilbert_tokenizer, framework='pt', device=device, return_all_scores=True)
camembert_classifier = pipeline('text-classification', model=camembert_model, tokenizer=camembert_tokenizer, framework='pt', device=device, return_all_scores=True)
flaubert_classifier = pipeline('text-classification', model=flaubert_model, tokenizer=flaubert_tokenizer, framework='pt', device=device, return_all_scores=True)

# Predict probabilities for the unlabelled data using both models
distilbert_probs = distilbert_classifier(unlabelled_data['sentence'].tolist())
camembert_probs = camembert_classifier(unlabelled_data['sentence'].tolist())
flaubert_probs = flaubert_classifier(unlabelled_data['sentence'].tolist())

# Convert the predictions to numpy arrays
distilbertprobs_array = np.array([[prob['score'] for prob in probs] for probs in distilbert_probs])
camembert_probs_array = np.array([[prob['score'] for prob in probs] for probs in camembert_probs])
flaubert_probs_array = np.array([[prob['score'] for prob in probs] for probs in flaubert_probs])

# Combine predictions using soft voting (average probabilities)
average_probs = (distilbertprobs_array + camembert_probs_array + flaubert_probs_array) / 3
final_preds = np.argmax(average_probs, axis=1)

# Decode the numeric labels to original labels using the loaded LabelEncoder

inverse_label_mapping = {v: k for k, v in label_mapping.items()}
final_labels = [inverse_label_mapping[pred] for pred in final_preds]

# Create a DataFrame to export
results_df = pd.DataFrame({
    'id': unlabelled_data['id'],
    'difficulty': final_labels
})

# Save the results to a new CSV file
results_df.to_csv('final.csv', index=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


# Predictions



In [ ]:
unlabelled_url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/unlabelled_test_data.csv"
unlabelled_data = pd.read_csv(unlabelled_url)

# Process unlabelled data
distilbert_encodings, camembert_encodings, features = process_data(unlabelled_data)

# Create datasets
distilbert_dataset = ClassificationDataset(distilbert_encodings, features)
camembert_dataset = ClassificationDataset(camembert_encodings, features)

# Set the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Make predictions with both models
models = [distilbert_model, camembert_model]
datasets = [distilbert_dataset, camembert_dataset]

preds_distilbert, preds_camembert = predict(models, datasets)

# Combine predictions (simple averaging in this case)
combined_preds = (preds_distilbert + preds_camembert) / 2
final_preds = np.argmax(combined_preds, axis=1)

# Map predictions back to difficulty levels
inverse_label_mapping = {v: k for k, v in label_mapping.items()}
final_labels = [inverse_label_mapping[pred] for pred in final_preds]

# Add predictions to the dataframe
unlabelled_data['predicted_difficulty'] = final_labels

In [9]:
submission = unlabelled_data[['id','predicted_difficulty']]

In [10]:
submission.to_csv('subs.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##TRY WITH AUGMENTED DATA GPT2

In [28]:
augmented_df = pd.read_csv('https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/augmented_dataset.csv')

In [29]:
def remove_duplicate_words(sentence):
  '''
  Function necessary to remove duplicated words in sentences generated by gpt-2 that made no sense.
  '''
  words = sentence.split()
  seen = set()
  unique_words = []
  for word in words:
      if word not in seen:
          unique_words.append(word)
          seen.add(word)
  return ' '.join(unique_words)

# Apply the function to the 'sentence' column
augmented_df.loc[4800:, 'sentence'] = augmented_df.loc[4800:, 'sentence'].apply(remove_duplicate_words)

In [30]:
# Function to fix the 'id' column
def fix_id_column(df):
    # Create a sequence of row numbers starting from 0
    correct_ids = pd.Series(range(len(df)))
    # Replace 'NaN' values and incorrect ids
    df['id'] = correct_ids
    return df

# Fix the 'id' column
augmented_df = fix_id_column(augmented_df)

In [31]:
import nltk
from nltk.corpus import wordnet
from random import randint
import random

nltk.download('wordnet')
nltk.download('omw-1.4')

def synonym_replacement(sentence, n):
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if wordnet.synsets(word)]))
    num_replaced = 0

    for random_word in random_word_list:
        synonyms = set()
        for syn in wordnet.synsets(random_word):
            for lemma in syn.lemmas():
                synonym = lemma.name().replace("_", " ").replace("-", " ").lower()
                synonyms.add(synonym)
        if len(synonyms) > 1:
            synonyms.discard(random_word)
            synonym = list(synonyms)[randint(0, len(synonyms) - 1)]
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:  # only replace up to n words
            break

    return ' '.join(new_words)

def shuffle_sentence(sentence):
    words = sentence.split()
    random.shuffle(words)
    return ' '.join(words)


augmented_data = []

# Augment data
for _, row in augmented_df.iterrows():
    original_sentence = row['sentence']
    difficulty = row['difficulty']

    # Generate augmented sentences
    augmented_sentence_synonym = synonym_replacement(original_sentence, 2)
    augmented_sentence_shuffled = shuffle_sentence(original_sentence) #idk


    # Append original and augmented sentences to the new list
    augmented_data.append({'sentence': original_sentence, 'difficulty': difficulty})
    augmented_data.append({'sentence': augmented_sentence_synonym, 'difficulty': difficulty})
    augmented_data.append({'sentence': augmented_sentence_shuffled, 'difficulty': difficulty})


# Create a new DataFrame from the augmented data
data = pd.DataFrame(augmented_data)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [32]:
# Map labels to integers
label_mapping = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}
data['difficulty'] = data['difficulty'].map(label_mapping)

# Feature Engineering Functions
def get_sentence_length(sentence):
    return len(sentence)

def get_word_count(sentence):
    words = nltk.word_tokenize(sentence)
    return len(words)

def get_avg_word_length(sentence):
    words = nltk.word_tokenize(sentence)
    return np.mean([len(word) for word in words])

def count_punctuation(sentence):
    return sum([1 for char in sentence if char in string.punctuation])

# Extract Features
data['sentence_length'] = data['sentence'].apply(get_sentence_length)
data['word_count'] = data['sentence'].apply(get_word_count)
data['avg_word_length'] = data['sentence'].apply(get_avg_word_length)
data['punctuation_count'] = data['sentence'].apply(count_punctuation)

# Standardize the features
scaler = StandardScaler()
feature_cols = ['sentence_length', 'word_count', 'avg_word_length', 'punctuation_count']
data[feature_cols] = scaler.fit_transform(data[feature_cols])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data, data['difficulty'], test_size=0.2, random_state=42)

# Tokenization
tokenizer = FlaubertTokenizer.from_pretrained('flaubert/flaubert_base_cased')

train_encodings = tokenizer(X_train['sentence'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(X_test['sentence'].tolist(), truncation=True, padding=True)

class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None, features=None):
        self.encodings = encodings
        self.labels = labels
        self.features = features

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        if self.features is not None:
            item['features'] = torch.tensor(self.features[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Set the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Load pre-trained model


# Cross-validation with KFold
kf = KFold(n_splits=3, shuffle=True, random_state=42)
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"Training fold {fold + 1}")

    # Prepare training and validation datasets
    train_data = data.iloc[train_idx]
    val_data = data.iloc[val_idx]

    model = FlaubertForSequenceClassification.from_pretrained('flaubert/flaubert_base_cased', num_labels=6).to(device)

    train_encodings_fold = tokenizer(train_data['sentence'].tolist(), truncation=True, padding=True)
    val_encodings_fold = tokenizer(val_data['sentence'].tolist(), truncation=True, padding=True)

    train_dataset = ClassificationDataset(train_encodings_fold, train_data['difficulty'].tolist(), train_data[feature_cols].values)
    val_dataset = ClassificationDataset(val_encodings_fold, val_data['difficulty'].tolist(), val_data[feature_cols].values)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f'./results_fold_{fold + 1}',
        num_train_epochs=8,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=1000,
        weight_decay=0.005,
        logging_dir=f'./logs_fold_{fold + 1}',
        logging_steps=20,
        evaluation_strategy="epoch",
        learning_rate=0.000015,
        fp16=True
    )

    # Define trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    predictions, label_ids, metrics = trainer.predict(val_dataset)
    preds = predictions.argmax(-1)

    # Calculate metrics
    accuracy = accuracy_score(val_data['difficulty'], preds)
    precision = precision_score(val_data['difficulty'], preds, average='weighted')
    recall = recall_score(val_data['difficulty'], preds, average='weighted')
    f1 = f1_score(val_data['difficulty'], preds, average='weighted')

    # Store metrics
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)

    # Print the results
    print(f'Fold {fold + 1} - Accuracy: {accuracy:.2f}')
    print(f'Fold {fold + 1} - Precision: {precision:.2f}')
    print(f'Fold {fold + 1} - Recall: {recall:.2f}')
    print(f'Fold {fold + 1} - F1 Score: {f1:.2f}')

# Print the average results across folds
print(f'Average Accuracy: {np.mean(accuracy_list):.2f}')
print(f'Average Precision: {np.mean(precision_list):.2f}')
print(f'Average Recall: {np.mean(recall_list):.2f}')
print(f'Average F1 Score: {np.mean(f1_list):.2f}')

# Alternatively, print a detailed classification report for the final fold
print(classification_report(val_data['difficulty'], preds, target_names=['A1', 'A2', 'B1', 'B2', 'C1', 'C2']))

Training fold 1


Some weights of FlaubertForSequenceClassification were not initialized from the model checkpoint at flaubert/flaubert_base_cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.113800,1.094316
2,0.727300,0.663986
3,0.421200,0.537530
4,0.490500,0.467832
5,0.224700,0.546166
6,0.280100,0.464303
7,0.204800,0.467088
8,0.114900,0.458993


Fold 1 - Accuracy: 0.94
Fold 1 - Precision: 0.94
Fold 1 - Recall: 0.94
Fold 1 - F1 Score: 0.94
Training fold 2


Some weights of FlaubertForSequenceClassification were not initialized from the model checkpoint at flaubert/flaubert_base_cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.101900,0.970492
2,0.655100,0.633322
3,0.330000,0.447865
4,0.157000,0.489751


KeyboardInterrupt: 